# Inspecting normalizations

Note: This notebook should only be used to generate predictions on git branches created from a dvc experiment. On the main/dev branch this notebook will only be updated when changes to the code are necessary, but it will not be used to generate predictions there.

## Prepare data and get normalizations

In [1]:
import os
import random
import tomli

import datasets
import numpy as np
import pandas as pd
import torch
import transformers
from transnormer.models.train_model import tokenize_datasets
from transnormer.evaluation.analysis import get_spans_of_unknown_tokens
from transnormer.visualization.formatting import markup_spans

/home/bracke/miniconda3/envs/gpu-venv-transnormer/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Number of examples to generate predictions for
N = 10

In [3]:
# Load configs
ROOT = "../../"
CONFIGFILE = os.path.join(ROOT, "models/models_2023-03-16_15-24/training_config.toml")
with open(CONFIGFILE, mode="rb") as fp:
    CONFIGS = tomli.load(fp)

# OR: Use custom configs (if so: uncomment the following)
# CONFIGS = {
#     "gpu": "cuda:0",
#     "random_seed": 42,
#     "tokenizer": {
#         "max_length_input": 128,
#         "max_length_output": 128,
#         "input_transliterator": "Transliterator1",
#     },
#     "language_models": {
#         "checkpoint_encoder": "prajjwal1/bert-tiny",
#         "checkpoint_decoder": "prajjwal1/bert-tiny",
#     },
#     "beam_search_decoding": {
#         "no_repeat_ngram_size": 3,
#         "early_stopping": True,
#         "length_penalty": 2.0,
#         "num_beams": 4,
#     },
# }

In [4]:
# Fix seeds for reproducibilty
random.seed(CONFIGS["random_seed"])
np.random.seed(CONFIGS["random_seed"])
torch.manual_seed(CONFIGS["random_seed"])

# GPU set-up
device = torch.device(CONFIGS["gpu"] if torch.cuda.is_available() else "cpu")

In [5]:
# Load data 
data_files = {
    "1600to1699": os.path.join(ROOT, "data/interim/dtak-v03-1600-1699/dtak-v03-1600-1699-train.jsonl"),
    # "1700to1799": os.path.join(ROOT, "data/interim/dtak-1700-1799/dtak-1700-1799-validation.jsonl"),
    # "1800to1899": os.path.join(ROOT, "data/interim/dtak-1800-1899/dtak-1800-1899-validation.jsonl"),
}
ds = datasets.load_dataset("json", data_files=data_files)

ds["1600to1699"] = ds["1600to1699"].shuffle().select(range(N))
# ds["1700to1799"] = ds["1700to1799"].shuffle().select(range(N))
# ds["1800to1899"] = ds["1800to1899"].shuffle().select(range(N))

Using custom data configuration default-0e8e502d3435a016
Found cached dataset json (/home/bracke/.cache/huggingface/datasets/json/default-0e8e502d3435a016/0.0.0/0f7e3662623656454fcd2b650f34e886a7db4b9104504885bd462096cc7a9f51)
100%|██████████| 1/1 [00:00<00:00, 30.77it/s]


In [6]:
# Tokenize data 

# In case we use locally saved models for the tokenizers
# the relative path must be completed. Uncomment the respective line.
# CONFIGS["language_models"]["checkpoint_encoder"] = os.path.join(ROOT, CONFIGS["language_models"]["checkpoint_encoder"])
# CONFIGS["language_models"]["checkpoint_decoder"] = os.path.join(ROOT, CONFIGS["language_models"]["checkpoint_decoder"])

prepared_dataset, tokenizer_input, tokenizer_output = tokenize_datasets(ds, CONFIGS)

Parameter 'fn_kwargs'={'tokenizer_input': BertTokenizerFast(name_or_path='dbmdz/bert-base-historic-multilingual-cased', vocab_size=32000, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}), 'tokenizer_output': BertTokenizerFast(name_or_path='bert-base-multilingual-cased', vocab_size=119547, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}), 'max_length_input': 128, 'max_length_output': 128} of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the

In [7]:
# Load model
checkpoint = os.path.join(ROOT, "models/models_2023-03-16_15-24/checkpoint-100000") # TODO
model = transformers.EncoderDecoderModel.from_pretrained(checkpoint).to(device)

In [8]:
# Generate normalizations
# TODO: Do we have to include a configuration for beam search decoding here?
def generate_normalization(batch):
    inputs = tokenizer_input(batch["orig"], padding="max_length", truncation=True, max_length=128, return_tensors="pt")
    input_ids = inputs.input_ids.to(device)
    attention_mask = inputs.attention_mask.to(device)

    outputs = model.generate(input_ids, attention_mask=attention_mask)
    output_str = tokenizer_output.batch_decode(outputs, skip_special_tokens=True)

    batch["norm_pred_str"] = output_str

    return batch


ds = ds.map(
    generate_normalization, 
    batched=True, 
    batch_size=8, 
    load_from_cache_file=False,
    )


  0%|          | 0/2 [00:00<?, ?ba/s]/home/bracke/miniconda3/envs/gpu-venv-transnormer/lib/python3.9/site-packages/transformers/generation/utils.py:1186: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation)
  warnings.warn(
/home/bracke/miniconda3/envs/gpu-venv-transnormer/lib/python3.9/site-packages/transformers/generation/utils.py:1273: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 128 (`generation_config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
100%|██████████| 2/2 [00:09<00:00,  4.7

### Apply visual modifications 



1. Markup unknown tokens in red
2. Add token-separator ("|") in input text
2. Add token-separator ("|") in output text

In [9]:
# Apply HTML markup to unknown tokens in original text
def markup_unknown_tokens(batch):
    spans = get_spans_of_unknown_tokens(batch["orig"], tokenizer_input)
    text_marked_up = markup_spans(batch["orig"], spans, opening_tag="<span style='color:#FF0000'>")

    batch["orig_marked_up"] = text_marked_up
    return batch

ds = ds.map(markup_unknown_tokens, batched=False, )

100%|██████████| 10/10 [00:00<00:00, 2021.84ex/s]


In [10]:
# Add token-separator 
def separate_tokens(batch, column, tokenizer):
    # We have to do the normalization explicitly before getting the encoding
    # to avoid mismatches in case the normalization changes the string length, e.g. "æ -> ae"
    norm_str = tokenizer.backend_tokenizer.normalizer.normalize_str(batch[column])
    encoding = tokenizer(norm_str, add_special_tokens=False)
    spans = [
        # map a token index to a pair of character indices
        encoding.token_to_chars(token_index)[:] for token_index in range(len(encoding["input_ids"]))
    ]
    text_marked_up = markup_spans(
        tokenizer.backend_tokenizer.normalizer.normalize_str(batch[column]),
        spans,
        opening_tag="",
        closing_tag="<span style='color:#FFA500'>|</span>",
        )

    batch[f"{column}_xlit_tok"] = text_marked_up
    return batch

ds = ds.map(separate_tokens, fn_kwargs={"tokenizer":tokenizer_input, "column" : "orig"}, batched=False, load_from_cache_file=False)
ds = ds.map(separate_tokens, fn_kwargs={"tokenizer":tokenizer_output, "column" : "norm_pred_str"}, batched=False, load_from_cache_file=False)

100%|██████████| 10/10 [00:00<00:00, 1271.77ex/s]


In [11]:
# Create pandas dataframes from predictions

# Do no truncate cells with long text
pd.set_option('display.max_colwidth', None)

part = "1600to1699"
df1600to1699 = pd.DataFrame(
    data={
        "orig_xlit" : ds[part]["orig_xlit_tok"], 
        "norm_tok" : ds[part]["norm_pred_str_xlit_tok"], 
        }
    )

# part = "1700to1799"
# df1700to1799 = pd.DataFrame(
#     data={
#         "orig_xlit" : ds[part]["orig_xlit_tok"], 
#         "norm_tok" : ds[part]["norm_pred_str_xlit_tok"], 
#         }
#     )

# part = "1800to1899"
# df1800to1899 = pd.DataFrame(
#     data={
#         "orig_xlit" : ds[part]["orig_xlit_tok"], 
#         "norm_tok" : ds[part]["norm_pred_str_xlit_tok"], 
#         }
#     )

---

## Look at the dataframes

### 1600 to 1699

In [12]:
from IPython.core.display import HTML
display(HTML(df1600to1699.head(N).to_html(escape=False)))

### 1700 to 1799

In [13]:
from IPython.core.display import HTML
display(HTML(df1700to1799.head(N).to_html(escape=False)))

NameError: name 'df1700to1799' is not defined

### 1800 to 1899

In [ ]:
from IPython.core.display import HTML
display(HTML(df1800to1899.head(N).to_html(escape=False)))

---

In [23]:
s_orig = "Nach etlichen Taͤntzen kam vnſer Præſidentin wider daher/ vnd vermeldet vns wie das ſich die Kuͤnſtler vnd Studioſi gegen jhrer Koͤnig. Majeſt. erbotten/ deren zu ehren/ vnd gefallen vor dero abzug/ ein froͤliche Comœdiam zu agieren/ wolten nuhn wir derſelben auch beywohnen/ vnd Koͤnig. Ma. auff der Sonnen Hauß begleiten/ daß were dero Lieb/ vnd wolte ſolches in allen gnaden erkennen:"
# s_orig = "Groß-Fuͤrſten in Finland/ Hertzogen zu Schonen/ Eheſten/ Lieffland/ Carelen/ Bremen/ Vehrden/ Stettin/ Pommern/ der Caſſuben und Wenden/"
s_norm = "Nach etlichen Tänzen kam unser Präsidentin wider daher/ und vermeldet uns wie das sich die Künstler und Studiosi gegen ihrer König. Majest. erboten/ deren zu ehren/ und gefallen vor deren Abzug/ ein fröhliche Comoediam zu agieren/ wollten nun wir derselben auch beiwohnen/ und König. Ma. auf der Sonnen Haus begleiten/ dass wäre deren Liebe/ und wollte solches in allen Gnaden erkennen:" 

# s_orig = "ſolen " #* 10
# s_norm = "Solen " #* 150
s_orig = "Weilen des menschen Lebent ist kürtz seine gedechtnüs Schwag, und die Sinne zum Misverstendtnüs stets geneiget (...)" 

inputs = tokenizer_input(s_orig, padding="max_length", truncation=True, max_length=256, return_tensors="pt")
outputs = tokenizer_output(s_norm, padding="max_length", truncation=True, max_length=256, return_tensors="pt")
input_ids = inputs.input_ids.to(device)
output_ids = outputs.input_ids.to(device)
output_ids[output_ids==0] = -100 # replace padding token with -100
attention_mask = inputs.attention_mask.to(device)
example = {
    "input_ids":input_ids,
    "attention_mask":attention_mask,
    "labels":output_ids
}

# Loss
output_forward = model(**example)

print(output_forward.loss)

# Generation
output_gen = model.generate(
    input_ids, 
    attention_mask=attention_mask, 
    max_length=256,
    no_repeat_ngram_size=3, # hier 0 zu setzen führt zu leerer Ausgabe (warum?)
    early_stopping=True,
    length_penalty=2.0,
    num_beams=3,
    )

outputs_as_str = tokenizer_output.batch_decode(output_gen, skip_special_tokens=True)
# print(outputs_as_str[0])

# Display output
from IPython.core.display import HTML
df_example = pd.DataFrame(data=outputs_as_str)
display(HTML(df_example.head(1).to_html(escape=False)))

/home/bracke/miniconda3/envs/gpu-venv-transnormer/lib/python3.9/site-packages/transformers/models/encoder_decoder/modeling_encoder_decoder.py:634: FutureWarning: Version v4.12.0 introduces a better way to train encoder-decoder models by computing the loss inside the encoder-decoder framework rather than in the decoder itself. You may observe training discrepancies if fine-tuning a model trained with versions anterior to 4.12.0. The decoder_input_ids are now created based on the labels, no need to pass them yourself anymore.
  warnings.warn(DEPRECATION_WARNING, FutureWarning)


tensor(10.1295, device='cuda:0', grad_fn=<NllLossBackward0>)


,0
0,"Weilen des Menschen Lebent ist kürz seine Gedächtnis Schwag, und die Sinne zum Missverstendtnis stets geneigt (... )"
